In [ ]:
%pip install pandas matplotlib
import subprocess
from pathlib import Path
import shutil
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

mpl.rcParams['figure.dpi'] = 300

In [ ]:
timeseries_intervals = [32, 64, 128]
ingest_rates = [10_000_000, 8_000_000, 6_000_000, 4_000_000, 2_000_000] # 0 to indicate no limit
path_to_graph, timestamp_position = "~/wikipedia-growth.txt", 2
path_to_project = "~/projects/lollipop"
path_to_go = shutil.which("go")
threads = 10 # 0 to use nproc

path_to_graph = Path(path_to_graph).expanduser()
path_to_project = Path(path_to_project).expanduser()
assert(path_to_graph.exists())
assert(path_to_project.exists())
assert(path_to_go)

In [ ]:
def create_results_directory():
    path_to_results = path_to_project / "results" / f"2023-sc-poster"
    path_to_results.mkdir(exist_ok=True)
    return path_to_results

(path_to_project / "results").mkdir(exist_ok=True)
results_dir = create_results_directory() # or replace with existing result folder

In [ ]:
class Algorithm:
    def __init__(self, name: str, graph: Path, results_dir: Path, timeseries_output: Path, timestamp_position: int, convert_to_undirected: bool = False, threads: int = 0) -> None:
        self.name = name
        self.code_path = path_to_project / "cmd" / f"lp-{name}"
        self.results_dir = results_dir
        self.convert_to_undirected = convert_to_undirected
        self.graph = graph
        self.timestamp_position = timestamp_position
        self.threads = threads
        self.timeseries_output = timeseries_output
        assert(self.graph.exists())
        assert(self.results_dir.exists())
        assert(self.code_path.exists())
        assert(self.timeseries_output.parent.exists())

    def run_timeseries(self, timeseries_interval: int, ingest_rate: int = 0, suffix: str = "", no_skip: bool = False) -> Path:
        path_timeseries = self.results_dir / f"timeseries-{self.name}-{timeseries_interval}-{ingest_rate}-{self.threads}{suffix}.csv"
        # path_timeseries = self.results_dir / f"timeseries-{self.name}-{timeseries_interval}-{ingest_rate}.csv"
        if (not no_skip) and path_timeseries.exists():
            print(f"Skipping as result exists: {path_timeseries}")
            return path_timeseries

        cmd = [path_to_go, "run", self.code_path, "-c", "-tquery", f"-pt={self.timestamp_position}", f"-g={self.graph}", f"-dt={timeseries_interval}"]
        if ingest_rate:
            cmd.append(f"-dr={ingest_rate}")
        if self.convert_to_undirected:
            cmd.append("-u")
        if self.threads:
            cmd.append(f"-t={self.threads}")
        
        print(f"Command: {cmd}")
        self.timeseries_output.unlink(missing_ok=True)
        process = subprocess.Popen(cmd, cwd=path_to_project)
        process.wait()

        assert(self.timeseries_output.exists())
        return self.timeseries_output.rename(path_timeseries)
    
pagerank = Algorithm(name="pagerank", graph=path_to_graph, results_dir=results_dir, 
                     timeseries_output=path_to_project / "results" / "timeseries.csv", 
                     timestamp_position=timestamp_position, 
                     convert_to_undirected=False, threads=threads)

colouring = Algorithm(name="colouring", graph=path_to_graph, results_dir=results_dir, 
                     timeseries_output=path_to_project / "results" / "colouring-timeseries.csv", 
                     timestamp_position=timestamp_position, 
                     convert_to_undirected=True, threads=threads)

In [ ]:
class Experiment:
    def __init__(self, results_dir: Path, algorithm: Algorithm) -> None:
        self.algorithm = algorithm
        self.results_dir = results_dir
        self.path_results_db = results_dir / "timeseries-db.csv"
        if self.path_results_db.exists():
            self.results_db = pd.read_csv(self.path_results_db)
        else:
            self.results_db = pd.DataFrame(columns=["Algorithm", "Query Interval", "Number of Queries", "Ingest Rate", "Average Latency", "Threads", "Timeseries Path"])
            self.results_db.to_csv(self.path_results_db, index=False)
        assert(self.results_dir.exists())

    def run(self, timeseries_intervals: list[int], ingest_rates: list[int], repeat: int=1) -> Path:
        for interval in timeseries_intervals:
            for rate in ingest_rates:
                for r in range(repeat):
                    timeseries = self.algorithm.run_timeseries(timeseries_interval=interval, ingest_rate=rate, suffix=f"-{r}")
                    df_timeseries = pd.read_csv(timeseries)
                    average_latency = df_timeseries.loc[:, 'qLatencyMS'].mean()
                    self.results_db.loc[len(self.results_db)] = {
                        "Algorithm": self.algorithm.name,
                        "Query Interval": interval,
                        "Number of Queries": df_timeseries.shape[0],
                        "Ingest Rate": rate,
                        "Average Latency": average_latency,
                        "Threads": self.algorithm.threads,
                        "Timeseries Path": timeseries
                    }
                    self.results_db.to_csv(self.path_results_db, index=False)
        return self.path_results_db

    def load_results(self) -> pd.DataFrame:
        return pd.read_csv(self.path_results_db)

pagerank_experiment = Experiment(results_dir, pagerank)
colouring_experiment = Experiment(results_dir, colouring)

In [ ]:
# pagerank_experiment.run(timeseries_intervals, ingest_rates)
# pagerank_results = pagerank_experiment.load_results()
# pagerank_results
colouring_experiment.run(timeseries_intervals, ingest_rates, repeat=5)
colouring_results = colouring_experiment.load_results()
colouring_results

In [ ]:
def get_average_latency(timeseries_paths: list[str]) -> pd.DataFrame:
    assert(timeseries_paths.shape[0] > 0)
    output_df = pd.read_csv(timeseries_paths.iloc[0])
    output_df = output_df[['ts', 'qLatencyMS']].copy()

    for path in timeseries_paths[1:]:
        df = pd.read_csv(path)
        assert((df['ts'] == output_df['ts']).all())
        output_df['qLatencyMS'] = output_df['qLatencyMS'].add(df['qLatencyMS'], fill_value=0)
    
    output_df['ts'] = pd.to_datetime(output_df['ts'])
    output_df['qLatencyMS'] = output_df['qLatencyMS'] / len(timeseries_paths)
    return output_df

import matplotlib.dates as mdates
def plot(timeseries_results, y_max, x_min):

    timeseries_path = timeseries_results.query(f'`Query Interval` == {min(timeseries_intervals)}').iloc[0]["Timeseries Path"]
    df_timeseries = pd.read_csv(timeseries_path)
    ts, e = pd.to_datetime(df_timeseries['ts']), df_timeseries['EC']
    min_ts, max_ts, max_e = min(ts), max(ts), max(e)
    print(f"min(ts)={min_ts} max(ts)={max_ts} e={e}")
    
    fig, axs = plt.subplots(len(timeseries_intervals), len(ingest_rates), sharex=True, figsize=(6, 4))

    for interval, axs_y in zip(timeseries_intervals, axs):
        for rate, ax in zip(ingest_rates, axs_y):
            ax = ax.twinx()
            if rate != ingest_rates[-1]:
                ax.set_yticks([])
            ax.set_zorder(1)
            ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
            ax.margins(x=0)
            ax.set_ylim([0, 1])
            ax.fill_between(ts, e / max_e, alpha=0.2, facecolor="green", edgecolor=None, label=f"% of edges added")

    for interval, axs_y in zip(timeseries_intervals, axs):
        for rate, ax in zip(ingest_rates, axs_y):
            timeseries_filtered = timeseries_results.query(f'`Query Interval` == {interval} and `Ingest Rate` == {rate} and `Threads` == {threads}')

            df_averaged = get_average_latency(timeseries_filtered["Timeseries Path"])
            x, y1 = df_averaged['ts'], df_averaged['qLatencyMS']

            ax.patch.set_visible(False)
            ax.margins(x=0)
            ax.set(xlim=[x_min, max_ts], ylim=[0,y_max], zorder=2, xticks=[])
            ax.plot(x, y1, marker = '.', markersize = 5, color="chocolate", label="Latency")
            ax.fill_between(x, y1, alpha=0.2, facecolor="red", edgecolor=None)

            if rate != ingest_rates[0]:
                ax.set_yticks([])

            if rate == ingest_rates[0] or (rate == ingest_rates[1] and interval <= 32):
                ax = ax.twinx().twiny() # don't share any axes
                ax.set(xlim=[0,1], ylim=[0,1], zorder=0, xticks=[], yticks=[])
                ax.scatter(0.2, 0.85, marker="x", color="brown", s=100, label="Target rate not achieved")

    for interval, ax in zip(timeseries_intervals, axs[:, 0]):
        ax.set_ylabel(f"{interval} days", labelpad=7)
    for rate, ax in zip(ingest_rates, axs[-1]):
        ax.set_xlabel(f"{rate/1e6}m e/s", labelpad=7)
    
    # Legend
    handles, labels = [sum(x, []) for x in zip(*[ax.get_legend_handles_labels() for ax in fig.axes])]
    handles_labels = dict(zip(labels, handles))
    fig.legend(handles_labels.values(), handles_labels.keys(), framealpha=0.9)
    
    fig.tight_layout(pad=0.5)
    return fig

fig = plot(colouring_results, 1000, pd.to_datetime("2005-03-15"))
fig.savefig(results_dir / "rate-limiting.pdf")